## 스케일링(scailing)
- 하는 이유 : 단위가 크면 해당 Weight 에 영향력이 크게 되므로 데이터의 분포만으로 가중치에 영향 주려고
- 대상 데이터 범위 조정해 모델 성능 개선
- 데이터 단위 줄여서 연산속도 에도 이득 가능 

- 지수화 와 유사
- 딥러닝에서는 정규화
- 데이터 형태에 따른 스케일링 방법이 다름.

In [69]:
# 개인적인 의문 : 단위가 크면 영향력이 커지는데 수학적으로 왜 그런지 알아보자

In [70]:
import numpy as np

data = np.array([[1,2,3],[4,5,6],[7,8,9]])

data_transform = np.array([[10,20,30],[40,50,60],[70,80,90]])

data_differents = np.array([[10,20,30],[400,500,600],[7000,8000,9000]]) 
# 단위가 다 달라도 비슷한 단위 분포로 조정해줌.

data_differents_n = np.array([[20,200,1000],[40,500,6000],[90,600,9000]]) 

data_differents_nn = np.array([ [1, 100, 0.001], # 작은 값과 매우 큰 값의 혼합 
          [50, 500, 0.005], 
          [100, 1000, 0.010], 
          [200, 2000, 0.020], 
          [400, 4000, 0.050] # 값이 점진적으로 증가 
          ]) 

### StandardScaler
- 평균 0, 표준편차가 1 로 저장하는 표준 정규분포
- 데이터가 정규분포를 따를 때 적합 (모든 컬럼이 정규분포 따라야함.)


In [71]:
from sklearn.preprocessing import StandardScaler
# 이것도 데이터 군집이라 교육을 시켜야함.
# 제 3의 데이터 군집을 학습된 분포에 맞추어 새로 스케일링 해 줌.

standardscaler = StandardScaler()
standardscaler.fit(data) # 스케일 학습
# fit 하고 transform 따로 해도 되고, fit_transform 같이 해도 됨.
standardscaler # 분포 공식 기억

StandardScaler()

In [72]:
standardscaler.transform(data) # 학습된 scail 분포로 대상의 분포를 변환

array([[-1.22474487, -1.22474487, -1.22474487],
       [ 0.        ,  0.        ,  0.        ],
       [ 1.22474487,  1.22474487,  1.22474487]])

In [73]:
standardscaler.fit(data_transform) # 스케일 학습


StandardScaler()

In [74]:
standardscaler.transform(data_transform) # 유사한 데이터 셋으로 학습해야함?

array([[-1.22474487, -1.22474487, -1.22474487],
       [ 0.        ,  0.        ,  0.        ],
       [ 1.22474487,  1.22474487,  1.22474487]])

In [75]:
standardscaler.fit(data_differents) # 스케일 학습

StandardScaler()

In [76]:
standardscaler.transform(data_differents) # 유사한 데이터 셋으로 학습해야함?


array([[-0.76703657, -0.77177189, -0.77546676],
       [-0.64543321, -0.64040646, -0.636468  ],
       [ 1.41246978,  1.41217836,  1.41193477]])

In [77]:
data_differents_n.shape

(3, 3)

In [78]:
standardscaler.fit(data_differents_n) # 스케일 학습

StandardScaler()

In [79]:
standardscaler.transform(data_differents_n) # 유사한 데이터 셋으로 학습해야함?

array([[-1.01904933, -1.37281295, -1.31319831],
       [-0.33968311,  0.39223227,  0.20203051],
       [ 1.35873244,  0.98058068,  1.1111678 ]])

In [80]:
data_differents_nn.shape

(5, 3)

In [81]:
standardscaler.fit(data_differents_nn) # 스케일 학습

StandardScaler()

In [82]:
standardscaler.transform(data_differents_nn) # 유사한 데이터 셋으로 학습해야함?

array([[-1.05658515, -1.01908043, -0.92104942],
       [-0.70958333, -0.73201552, -0.69362981],
       [-0.35549983, -0.37318438, -0.4093553 ],
       [ 0.35266716,  0.34447789,  0.15919373],
       [ 1.76900115,  1.77980245,  1.86484081]])

In [83]:
# 배포 및 서비스
# 이대로 배포하면 사용자가 변환된 값을 넣어야함.
# input : 50,500,0.005

standardscaler.transform([[50,500,0.005]]) 
# model 있으면 predict를 한다. 
# 배포 시에 모델과 스케일러를 같이 배포해야해


array([[-0.70958333, -0.73201552, -0.69362981]])

In [84]:
# 다시 되돌릴수도 있다. 물론 오차 있음.
standardscaler.inverse_transform([[-0.70958333, -0.73201552, -0.69362981]])

array([[4.99999995e+01, 5.00000003e+02, 5.00000005e-03]])

## Minmax scale
- 특정 데이터를 특정 범위 (0~1) 변환
- 데이터가 고르게 분포 되어 있는 경우 사용
- image 처리에서 주로 사용

In [85]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
minmaxscaler.fit(data_differents_nn)

MinMaxScaler()

In [86]:
minmaxscaler.transform(data_differents_nn)

array([[0.        , 0.        , 0.        ],
       [0.12280702, 0.1025641 , 0.08163265],
       [0.2481203 , 0.23076923, 0.18367347],
       [0.49874687, 0.48717949, 0.3877551 ],
       [1.        , 1.        , 1.        ]])

## Maxabsscale
- 각 특성(feature) 절대값 최대치를 1로 스케일, 음수 값을 유지 하는게 특징
- 희소 행렬(0이 많은 행렬) 처리 적합
- 일반적으론 잘 안쓰나봄

In [87]:
from sklearn.preprocessing import MaxAbsScaler
maxabsscaler = MaxAbsScaler()
maxabsscaler.fit(data_differents_nn)

MaxAbsScaler()

In [88]:
maxabsscaler.transform(data_differents_nn)


array([[0.0025, 0.025 , 0.02  ],
       [0.125 , 0.125 , 0.1   ],
       [0.25  , 0.25  , 0.2   ],
       [0.5   , 0.5   , 0.4   ],
       [1.    , 1.    , 1.    ]])

## Robustscale
- 중앙값과 IQR 사용한 변환
- 이상치 데이터 많을 때 사용
- 일일히 컬럼마다 이상치 조절하기 힘들 때

In [89]:
from sklearn.preprocessing import RobustScaler
robustscaler = RobustScaler()
robustscaler.fit(data_differents_nn)

RobustScaler()

In [90]:
robustscaler.transform(data_differents_nn)

array([[-0.66      , -0.6       , -0.6       ],
       [-0.33333333, -0.33333333, -0.33333333],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.66666667,  0.66666667,  0.66666667],
       [ 2.        ,  2.        ,  2.66666667]])

## Normalizer
- 벡터의 크기를 1로 만드는 정규화 변환
- 벡터간의 거리 기반 계산 적합(DL 에서 주로 씀)

In [91]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
normalizer.fit(data_differents_nn)

Normalizer()

In [92]:
normalizer.transform(data_differents_nn)

array([[9.99950004e-03, 9.99950004e-01, 9.99950004e-06],
       [9.95037190e-02, 9.95037190e-01, 9.95037190e-06],
       [9.95037190e-02, 9.95037190e-01, 9.95037190e-06],
       [9.95037190e-02, 9.95037190e-01, 9.95037190e-06],
       [9.95037190e-02, 9.95037190e-01, 1.24379649e-05]])